In [55]:
import eli5
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import hstack

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Imputer

from time import time

## Data Preprocessing

In [59]:
# read data from 2 files into pandas dataframes
df1 = pd.read_csv('winemag-data_first150k.csv')
df2 = pd.read_csv('winemag-data-130k-v2.csv')
# concatenate the 2 dataframes
data = pd.concat([df1, df2], ignore_index=True)
print('The raw data has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

The raw data has 280901 rows and 14 columns


In [60]:
# drop columns only applies to one of the two dataframes
data = data.drop(['taster_name', 'taster_twitter_handle', 'title', 'designation'], axis=1)
# drop index columns
data = data.drop(labels='Unnamed: 0', axis=1)
# deduplicate data
data_deduped = data.drop_duplicates()
# the missing value percentage for the remaining columns
data_deduped.isnull().mean()

country        0.000352
description    0.000000
points         0.000000
price          0.075287
province       0.000352
region_1       0.163981
region_2       0.603278
variety        0.000006
winery         0.000000
dtype: float64

In [5]:
# drop region_2 because missing value rate is too high
data_deduped = data.drop(['region_2'], axis=1)
# fill missing values with 'NaN'. this prevents later issue in tf-idf.
data_deduped = data_deduped.fillna('NaN')
# data shape
data_deduped.shape
print('The deduped data has {} rows and {} columns'.format(data_deduped.shape[0], data_deduped.shape[1]))

The deduped data has 170520 rows and 8 columns


In [6]:
# create a tf-idf vectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='ascii',
    analyzer='word',
    stop_words='english',
    max_df=0.1)

In [7]:
# vectorize description using tf-idf
descr_clean = word_vectorizer.fit_transform(data_deduped['description'])

In [8]:
# use one hot encoding to vectorize province
province_int = LabelEncoder().fit_transform(data_deduped['region_1'])
province_clean = Imputer().fit_transform(province_int.reshape(-1, 1))
province_clean = OneHotEncoder().fit_transform(province_clean)

In [9]:
# use one hot encoding to vectorize variety
variety_int = LabelEncoder().fit_transform(data_deduped['variety'])
variety_clean = Imputer().fit_transform(variety_int.reshape(-1, 1))
variety_clean = OneHotEncoder().fit_transform(variety_clean)

In [10]:
# scale price
price_clean = Imputer().fit_transform(data['price'].reshape(-1, 1))
price_clean = MinMaxScaler().fit_transform(price_clean)

/Users/laralu/anaconda2/envs/data_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [11]:
data_clean = hstack([descr_clean, province_clean, price_clean])

In [12]:
print('The proprocessed data has {} rows and {} columns'.format(data_clean.shape[0], data_clean.shape[1]))

The proprocessed data has 170520 rows and 43065 columns


## Model Training

In [13]:
# split the training and testing set
xtrain, xtest, ytrain, ytest = train_test_split(
    data_clean, data['points'], random_state=42)

In [14]:
# train SGD regressor using training data
sgd_cls = SGDRegressor(max_iter=100, learning_rate='constant', random_state=42)
sgd_cls.fit(xtrain, ytrain)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='squared_loss', max_iter=100, n_iter=None, penalty='l2',
       power_t=0.25, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [15]:
# predict test data
ypredict_sgd = sgd_cls.predict(xtest)

In [16]:
# calculate R2 score of test prediction
sgd_r2 = r2_score(ytest, ypredict_sgd)
print("The r2 score of SGD Regressor is {}".format(sgd_r2))

The r2 score of SGD Regressor is 0.690401098506


## Tuning

In [17]:
sgd_t1 = time()
# use grid search to find the best SGD regressor hyper parameters
param_grid = [
  {'max_iter': [20, 50, 80, 100],
   'penalty': ['l2', 'l1'],
   'learning_rate': ['constant', 'optimal', 'invscaling'],
   'power_t': [0.15, 0.25, 0.3]
  }
 ]
regressor = GridSearchCV(SGDRegressor(random_state=42, epsilon=0.1), param_grid)
regressor.fit(xtrain, ytrain)
sgd_t2 = time()

In [18]:
# avergae train time
print('SGD regressor training on average takes {}'.format((sgd_t2-sgd_t1)/72))

SGD regressor training on average takes 5.49731722474


In [19]:
regressor.best_estimator_

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='squared_loss', max_iter=50, n_iter=None, penalty='l2',
       power_t=0.15, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [20]:
sgd_r2 = regressor.best_estimator_.score(xtest, ytest)
print("The r2 score of SGD Regressor is {}".format(sgd_r2))

The r2 score of SGD Regressor is 0.690761804792


## Benchmark

In [21]:
lr_t1 = time()
# train a linear regressor
lr_cls = LinearRegression()
lr_cls.fit(xtrain, ytrain)
lr_t2 = time()
# train time
print('Linear regression training takes {}'.format(lr_t2-lr_t1))

Linear regression training takes 94.7833402157


In [22]:
ypredict_lr = lr_cls.predict(xtest)

In [23]:
lr_r2 = r2_score(ytest, ypredict_lr)
print("The r2 score of Linear Regressor is {}".format(lr_r2))

The r2 score of Linear Regressor is 0.66086119742


## Different Splits

In [24]:
# use different random states to control different random splits of training and testing sets
random_states = [12, 24, 37, 45, 53, 67, 77, 89, 91, 100]
r2_list = []
for rs in random_states:
    xtrain, xtest, ytrain, ytest = train_test_split(data_clean, data['points'], random_state=rs)
    r = SGDRegressor(max_iter=regressor.best_params_['max_iter'],
                    penalty=regressor.best_params_['penalty'],
                    learning_rate=regressor.best_params_['learning_rate'],
                    power_t=regressor.best_params_['power_t'],
                    random_state=rs)
    r.fit(xtrain, ytrain)
    ypredict_r = r.predict(xtest)
    r2 = r2_score(ytest, ypredict_r)
    r2_list.append(r2)
print('Average R2 score of SGD Regressor: {}'.format(np.average(r2_list)))
print('Standard deviation of R2 score of SGD Regressor: {}'.format(np.std(r2_list)))

Average R2 score of SGD Regressor: 0.690627946171
Standard deviation of R2 score of SGD Regressor: 0.00250053387275


## Visualization

In [52]:
# only care about description features, pad the rest of the feature names with 'N/A'
fill_length = data_clean.shape[1]-len(word_vectorizer.get_feature_names())
feature_names = word_vectorizer.get_feature_names() + ['N/A'] * fill_length
# show weights of each feature, exclude 'N/A' (non-description features)
eli5.show_weights(
    regressor.best_estimator_,
    feature_names = feature_names,
    feature_filter=(lambda x: x!='N/A'),
    top=(20, 20))

Weight?,Feature
+85.774,<BIAS>
+6.259,gorgeous
+5.638,beautiful
+5.356,long
+4.902,impressive
+4.855,complex
+4.835,beautifully
+4.574,years
+4.574,2020
+4.511,2025


In [53]:
eli5.show_weights(
    lr_cls,
    feature_names = feature_names,
    feature_filter=(lambda x: x!='N/A'),
    top=(20, 20))

Weight?,Feature
+361.478,ramandolo
+88.242,<BIAS>
+43.488,liguria
+29.938,safeguarding
+28.063,sebella
+27.412,markings
+26.767,spelunking
+26.599,flagvery
+26.504,16g
+25.580,vivre
